<a href="https://colab.research.google.com/github/ajibigad/ML-Playground/blob/main/Spam_SMS_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from fastai.text.all import *
import kagglehub

from pathlib import Path

# Define the paths
# Dataset can be found here https://www.kaggle.com/datasets/mariumfaheem666/spam-sms-classification-using-nlp/data

# , path="Spam_SMS.csv"

path = kagglehub.dataset_download("mariumfaheem666/spam-sms-classification-using-nlp", path="Spam_SMS.csv")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/mariumfaheem666/spam-sms-classification-using-nlp/versions/1/Spam_SMS.csv


In [23]:
categories = []
messages = []

with open(path, 'r') as file:
    lines = file.readlines()

print(lines[0])

with open(path, 'r', encoding='utf-8-sig') as csvfile:
  reader = csv.DictReader(csvfile)
  for row in reader:
    print(row)
    categories.append(row['Class'])
    messages.append(row['Message'])

Streaming output truncated to the last 5000 lines.
{'Class': 'ham', 'Message': 'Waiting for your call.'}
{'Class': 'ham', 'Message': 'Nope i waiting in sch 4 daddy...'}
{'Class': 'spam', 'Message': 'You have won ?1,000 cash or a ?2,000 prize! To claim, call09050000327'}
{'Class': 'ham', 'Message': "I'm tired of arguing with you about this week after week. Do what you want and from now on, i'll do the same."}
{'Class': 'ham', 'Message': 'Ü wait 4 me in sch i finish ard 5..'}
{'Class': 'spam', 'Message': 'our mobile number has won £5000, to claim calls us back or ring the claims hot line on 09050005321.'}
{'Class': 'ham', 'Message': 'Arngd marriage is while u r walkin unfortuntly a snake bites u. bt love marriage is dancing in frnt of d snake &amp; sayin Bite me, bite me.'}
{'Class': 'ham', 'Message': 'Huh so early.. Then ü having dinner outside izzit?'}
{'Class': 'ham', 'Message': 'Ok anyway no need to change with what you said'}
{'Class': 'spam', 'Message': 'We tried to contact you re 

In [24]:
len(categories)

5574

In [25]:
txt = messages[0]
print(txt)

spacy = WordTokenizer()
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


(#28) ['xxbos','xxmaj','go','until','jurong','point',',','crazy','..','xxmaj','available','only','in','bugis','n','great','world','la','e','buffet','…','xxmaj','cine','there','got','amore','wat','…']


In [26]:
txts = L(messages)
toks200 = txts[:200].map(tkn)
toks200[2]

(#38) ['xxbos','xxmaj','free','entry','in','2','a','wkly','comp','to'...]

In [27]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#296) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','.','i','you','to',',','?','a','the','!','…','and'...]"

In [28]:
nums = num(toks200[2]); nums
' '.join(num.vocab[o] for o in nums)

"xxbos xxmaj free xxunk in 2 a xxunk xxunk to win xxup xxunk xxmaj xxunk xxunk xxunk xxunk xxmaj xxunk xxunk . xxmaj text xxup xxunk to xxunk to receive xxunk xxunk txt xxunk 's xxunk xxunk 's"

In [29]:
nums200 = toks200.map(num)
dl = LMDataLoader(nums200)

x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [30]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj go until xxunk xxunk , xxunk .. xxmaj xxunk only in xxunk n great xxunk xxunk e xxunk'

In [31]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj go until xxunk xxunk , xxunk .. xxmaj xxunk only in xxunk n great xxunk xxunk e xxunk …'

In [32]:
toks = txts.map(tkn)
num = Numericalize()
num.setup(toks)

In [33]:
for i, msg in enumerate(messages):
  with open('./all_texts/' + f'{i}.txt', 'w') as f:
      f.write(msg)

In [34]:
path = Path('./all_texts')
dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_text_files, splitter=RandomSplitter(0.2)
).dataloaders(path, path=path, bs=128, seq_len=80)
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj now press xxunk da :) xxbos xxmaj so many people seems to be special at first sight , xxmaj but only very few will remain special to you till your last sight .. xxmaj maintain them till life ends .. xxmaj xxunk xxbos xxmaj ur cash - balance is currently 500 pounds - to maximize ur cash - in now send xxup go to 86688 only 150p / msg . xxup cc : 08718720201 xxup po xxup box","xxmaj now press xxunk da :) xxbos xxmaj so many people seems to be special at first sight , xxmaj but only very few will remain special to you till your last sight .. xxmaj maintain them till life ends .. xxmaj xxunk xxbos xxmaj ur cash - balance is currently 500 pounds - to maximize ur cash - in now send xxup go to 86688 only 150p / msg . xxup cc : 08718720201 xxup po xxup box 114"
1,will xxunk my promise . xxmaj you have been wonderful and a xxunk at all times . xxbos xxmaj send a logo 2 ur lover - 2 names joined by a heart . xxmaj txt xxup love xxup xxunk xxup xxunk xxup xxunk eg xxup love xxup xxunk xxup eve xxunk to 87077 xxmaj yahoo ! pobox36504w45wq xxunk 4 no xxunk 150p . xxbos xxmaj just got outta class gon na go gym . xxbos :( but your not here,xxunk my promise . xxmaj you have been wonderful and a xxunk at all times . xxbos xxmaj send a logo 2 ur lover - 2 names joined by a heart . xxmaj txt xxup love xxup xxunk xxup xxunk xxup xxunk eg xxup love xxup xxunk xxup eve xxunk to 87077 xxmaj yahoo ! pobox36504w45wq xxunk 4 no xxunk 150p . xxbos xxmaj just got outta class gon na go gym . xxbos :( but your not here …


In [35]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3,
    metrics=[accuracy, Perplexity()]).to_fp16()

learn.fit_one_cycle(1, 2e-2)
learn.unfreeze()
learn.fit_one_cycle(6, 2e-3)
learn.save_encoder('finetuned')

/usr/local/lib/python3.10/dist-packages/fastai/text/learner.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(wgts_fname, map_location = lambda storage

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,5.257209,4.688397,0.221115,108.678871,00:06


/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.519612,4.349571,0.245554,77.445213,00:04
1,4.310522,4.052417,0.279298,57.536369,00:08
2,4.059983,3.863014,0.304846,47.608650,00:04
3,3.820818,3.789731,0.319649,44.244488,00:07
4,3.626722,3.770757,0.321025,43.412914,00:07
5,3.467580,3.769880,0.324983,43.374863,00:06


In [36]:
def get_cat(path):
  idx = int(path.stem)
  return categories[idx]

dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab), CategoryBlock),
    get_y = get_cat,
    get_items=get_text_files,
    splitter=RandomSplitter(0.2)
).dataloaders(path, path=path, bs=128, seq_len=72)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [37]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj the last thing i ever wanted to do was hurt you . xxmaj and i did n't think it would have . xxmaj you 'd laugh , be embarassed , xxunk the xxunk and keep going . xxmaj but as far as i knew , it was n't even up . xxmaj the fact that you even felt like i would do it to hurt you shows you really do n't know me at all . xxmaj it was xxunk wednesday , but it was n't bad . xxmaj the problem i have with it is you xxup have the time to clean it , but you choose not to . xxmaj you skype , you take pictures , you sleep , you want to go out . i do n't mind a few things here and there , but when you do n't make the bed ,",ham
1,xxbos xxmaj for me the love should start with xxunk should feel that i need her every time around xxunk should be the first thing which comes in my xxunk would start the day and end it with xxunk should be there every time i xxunk will be then when my every xxunk has her xxunk life should happen around xxunk life will be xxunk to xxunk would cry for xxunk give all my happiness and take all her xxunk will be ready to fight with anyone for xxunk will be in love when i will be doing the xxunk things for xxunk will be when i do n't have to xxunk anyone that my girl is the most beautiful lady on the whole xxunk will always be xxunk xxunk for xxunk will be when i start up making chicken xxunk and end up xxunk xxunk will be the most,ham
2,"xxbos < # > is fast xxunk . xxmaj so , xxmaj wish u a very xxmaj happy xxmaj new xxmaj year xxmaj happy xxmaj xxunk xxmaj happy xxunk day xxmaj happy xxmaj valentines xxmaj day xxmaj happy xxmaj xxunk xxmaj happy xxmaj xxunk xxmaj happy xxmaj xxunk day xxmaj happy xxmaj may xxmaj day xxmaj happy xxmaj xxunk xxmaj day , xxmaj happy friendship , mother , father , xxunk , xxunk xxmaj day , & & xxup happy xxup birthday 4 xxup u. xxmaj happy xxmaj xxunk xxunk xxmaj happy xxmaj xxunk xxmaj happy xxmaj xxunk xxmaj happy xxmaj christmas < # > xxmaj good xxmaj xxunk xxmaj xxunk , xxmaj xxunk xxmaj nights . xxunk xxup am the first to xxup wishing u xxup all xxup these … your 's xxmaj xxunk",ham


In [38]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5,
                                metrics=accuracy).to_fp16()
learn = learn.load_encoder('finetuned')

/usr/local/lib/python3.10/dist-packages/fastai/text/learner.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(wgts_fname, map_location = lambda storage

In [39]:
learn.fit_one_cycle(1, 2e-2)
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,time
0,0.456195,0.185917,0.973070,00:05


/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,time
0,0.236611,0.221925,0.969479,00:07


/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,time
0,0.128169,0.191664,0.973968,00:05


/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,time
0,0.079200,0.194491,0.973070,00:06
1,0.064705,0.126848,0.975763,00:07


In [51]:
learn.predict("Congratulations! You've won a $1,000 Walmart gift card. Go to http://bit.ly/123456 tp claim now.")

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


('spam', tensor(1), tensor([0.0045, 0.9955]))

In [44]:
interp = Interpretation.from_learner(learn)
interp.plot_top_losses(9)

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


,input,target,predicted,probability,loss
0,"xxbos xxmaj hey xxmaj boys . xxmaj want hot xxrep 3 x pics sent direct 2 ur phone ? xxmaj txt xxup porn to xxunk , xxunk free and then just 50p per day . xxmaj to stop text xxup xxunk xxup xxunk xxup wc1n3xx",spam,spam,0.9986249208450317,5.315474033355713
1,xxbos xxmaj new xxup xxunk xxmaj chat 2 horny guys in ur area 4 just 25p xxmaj free 2 receive xxmaj search postcode or at xxunk . xxup txt xxup one name to xxunk . xxunk xxrep 3 0 xxunk xxunk xxmaj stop 2 xxunk,spam,spam,0.9998706579208374,5.315474033355713
2,xxbos xxmaj xxunk xxmaj number : xxunk - xxup ur awarded a xxmaj city xxmaj break and could xxup win a £ 200 xxmaj summer xxmaj shopping spree every xxup wk . xxmaj txt xxup store to 88039 . skilgme . xxunk £ xxunk,spam,spam,0.997285008430481,5.315474033355713
3,"xxbos freemsg xxmaj you have been awarded a xxup free mini xxup digital xxup camera , just reply xxup xxunk to collect your prize ! ( xxunk xxmaj opt out ? xxmaj stop xxunk / wk xxup sp : xxunk xxmaj xxunk )",spam,spam,0.9992114305496216,5.315474033355713
4,xxbos xxmaj great xxmaj news ! xxmaj call xxup freefone xxunk xxrep 4 4 7 to claim your guaranteed £ 1 xxrep 3 0 xxup cash or £ 2 xxrep 3 0 gift . xxmaj speak to a live operator xxup now !,spam,spam,0.9989866614341736,5.315474033355713
5,"xxbos xxmaj this is the 2nd time we have tried 2 contact u. u have won the 750 xxmaj pound prize . 2 claim is easy , call xxunk xxup now ! xxmaj only 10p per min . bt - national - rate",spam,spam,0.9987840056419373,5.315474033355713
6,xxbos xxmaj burger xxmaj king - xxmaj wanna play xxunk at a top xxunk ? xxmaj get 2 xxmaj burger xxmaj king before 1st xxmaj sept and go xxmaj xxunk or xxmaj super with xxunk - xxunk and walk out a winner,spam,ham,0.91804039478302,5.315474033355713
7,xxbos xxmaj sunshine xxmaj quiz xxmaj wkly xxup q ! xxmaj win a top xxmaj sony xxup dvd player if u know which country xxmaj liverpool played in mid week ? xxmaj txt ansr to 82277 . £ 1.50 sp : tyrone,spam,spam,0.9980429410934448,5.315474033355713
8,xxbos xxmaj we tried to contact you re our offer of xxmaj new xxmaj video xxmaj phone 750 anytime any network mins xxup half xxup price xxmaj rental camcorder call 08 xxrep 3 0 930705 or reply for delivery xxmaj we d,spam,spam,0.9968288540840149,5.315474033355713
